## Probability a Single Facility in a Single Year Seeds a Pandemic:

$
P_{\text{single\_pandemic}} = P_{\text{release}} \times P_{\text{seeds\_pandemic}}
$

where:
- $P_{\text{release}}$ is the probability of community release from a single facility in a single year.
- $P_{\text{seeds\_pandemic}}$ is the probability that a virus release seeds a pandemic. Since $P_{\text{seeds\_pandemic}}$ is given as a range, we will sample from a uniform distribution between 0.05 and 0.4.

## Number of Pandemics Seeded by Any matHPAI Facility:

For each year, for each facility, we'll calculate the probability it seeds a pandemic using a Bernoulli trial with $P_{\text{single\_pandemic}}$. Summing across all facilities and all years will give us the total number of pandemics seeded by any facility.

## Worldwide Fatalities:

For each pandemic, the number of fatalities is calculated as:

$
\text{fatalities} = \text{population} \times \text{infection\_rate} \times \text{fatality\_rate}
$

Summing across all pandemics will give the total fatalities.

In [1]:
from ipywidgets import interact, FloatSlider, IntSlider
import numpy as np
import plotly.graph_objects as go

#################################
# Define parameters and functions
#################################

# Given parameters
P_release = 0.00246
P_seeds_pandemic_min = 0.05
P_seeds_pandemic_max = 0.4
num_years = 100
num_facilities = 14
population = 7.9e9
fatality_rate = 0.025
infection_rate = 0.15

# Monte Carlo simulations
num_simulations = 1000

def run_simulation(
        P_release=0.00246,
        P_seeds_pandemic_min=0.05,
        P_seeds_pandemic_max=0.4,
        num_years = 100,
        num_facilities = 14,
        population = 7.9e9,
        fatality_rate = 0.025,
        infection_rate = 0.15,
        num_simulations = 100000
    ):
    """
    Run a Monte Carlo simulation of the number of pandemics and fatalities
    """
    total_pandemics = []
    total_fatalities = []

    for _ in range(num_simulations):
        P_seeds_pandemic = np.random.uniform(P_seeds_pandemic_min, P_seeds_pandemic_max)
        P_single_pandemic = P_release * P_seeds_pandemic
        
        # Number of pandemics seeded by any facility in a single year
        pandemics_per_year = np.random.binomial(num_facilities, P_single_pandemic, num_years)
        total_num_pandemics = sum(pandemics_per_year)
        
        # Fatalities for each pandemic
        fatalities_per_pandemic = population * infection_rate * fatality_rate * total_num_pandemics
        
        total_pandemics.append(total_num_pandemics)
        total_fatalities.append(fatalities_per_pandemic)

    # Convert to numpy arrays for statistics
    total_pandemics = np.array(total_pandemics)
    total_fatalities = np.array(total_fatalities)

    # Calculate some statistics
    mean_pandemics = np.mean(total_pandemics)
    mean_fatalities = np.mean(total_fatalities)
    percentile_5_fatalities = np.percentile(total_fatalities, 5)
    percentile_95_fatalities = np.percentile(total_fatalities, 95)

    plot_histogram(total_fatalities, mean_fatalities, np.median(total_fatalities), percentile_5_fatalities, percentile_95_fatalities)

    return mean_pandemics, mean_fatalities, percentile_5_fatalities, percentile_95_fatalities

def plot_histogram(total_deaths, mean_deaths, median_deaths, percentile_5, percentile_95):
    """Plot an interactive histogram of expected deaths with mean, median, and percentiles."""

    # Create the interactive histogram using plotly
    fig = go.Figure()

    # Add histogram data
    fig.add_trace(go.Histogram(x=total_deaths, name='Deaths Distribution', 
                               xbins=dict(size=5e7), # bin size of 50 million
                               marker_color='skyblue'))

    # Add vertical lines for mean, median, and percentiles
    fig.add_shape(
        go.layout.Shape(type='line', x0=mean_deaths, x1=mean_deaths, y0=0, y1=1, yref='paper',
                        line=dict(color='red', dash='dash'))
    )
    fig.add_shape(
        go.layout.Shape(type='line', x0=median_deaths, x1=median_deaths, y0=0, y1=1, yref='paper',
                        line=dict(color='green', dash='dash'))
    )
    fig.add_shape(
        go.layout.Shape(type='line', x0=percentile_5, x1=percentile_5, y0=0, y1=1, yref='paper',
                        line=dict(color='blue', dash='dash'))
    )
    fig.add_shape(
        go.layout.Shape(type='line', x0=percentile_95, x1=percentile_95, y0=0, y1=1, yref='paper',
                        line=dict(color='yellow', dash='dash'))
    )

    # Add legend and labels
    fig.update_layout(
        title="Distribution of Expected Deaths from Accidental Pandemics This Century",
        xaxis_title="Number of Deaths",
        yaxis_title="Count",
        shapes=[
            dict(type='line', xref='x', yref='paper',
                 x0=mean_deaths, y0=0, x1=mean_deaths, y1=1, line=dict(color='red', dash='dash'),
                 name='Mean'),
            dict(type='line', xref='x', yref='paper',
                 x0=median_deaths, y0=0, x1=median_deaths, y1=1, line=dict(color='green', dash='dash'),
                 name='Median'),
            dict(type='line', xref='x', yref='paper',
                 x0=percentile_5, y0=0, x1=percentile_5, y1=1, line=dict(color='blue', dash='dash'),
                 name='5th Percentile'),
            dict(type='line', xref='x', yref='paper',
                 x0=percentile_95, y0=0, x1=percentile_95, y1=1, line=dict(color='yellow', dash='dash'),
                 name='95th Percentile')
        ],
        legend_title_text='Statistics'
    )

    fig.show()

In [2]:
interact(run_simulation, 
        P_release=FloatSlider(min=0, max=0.01, step=0.0001, value=0.00246),
        P_seeds_pandemic_min=FloatSlider(min=0, max=1, step=0.01, value=0.05),
        P_seeds_pandemic_max=FloatSlider(min=0, max=1, step=0.01, value=0.4),
        num_years=IntSlider(min=0, max=100, step=1, value=100),
        num_facilities=IntSlider(min=0, max=100, step=1, value=14),
        population=IntSlider(min=0, max=1e10, step=1e8, value=7.9e9),
        fatality_rate=FloatSlider(min=0, max=1, step=0.01, value=0.025),
        infection_rate=FloatSlider(min=0, max=1, step=0.01, value=0.15),
        num_simulations=IntSlider(min=0, max=100000, step=1000, value=100000)
        )

interactive(children=(FloatSlider(value=0.00246, description='P_release', max=0.01, step=0.0001), FloatSlider(…

<function __main__.run_simulation(P_release=0.00246, P_seeds_pandemic_min=0.05, P_seeds_pandemic_max=0.4, num_years=100, num_facilities=14, population=7900000000.0, fatality_rate=0.025, infection_rate=0.15, num_simulations=100000)>